<a href="https://colab.research.google.com/github/PrajaktaPatil1/benign_malignant/blob/master/benign_malignant(with_GLCM_feature).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt   # importing lilbries
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image

In [2]:
import os
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
import cv2
from skimage.feature import greycomatrix, greycoprops

In [4]:
from google.colab import files
uploaded = files.upload()

Saving archive (update1).zip to archive (update1).zip


In [5]:
import os
import zipfile

# Path to the zip file
zip_file_path = '/content/archive (update1).zip'

# Directory where you want to extract the files
extracted_dir = '/content/extracted_data'

# Ensure the directory exists
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# List the contents of the extracted directory
extracted_files = os.listdir(extracted_dir)
print("Contents of the extracted directory:")
for file_name in extracted_files:
    print(file_name)

# Destination directory to save the extracted files
save_dir = '/content/saved_files'

Contents of the extracted directory:
archive (4)


##Data Preprocessing and Feature Extraction

#Gray-Level Co-occurrence Matrix

In [6]:
# Step 1: Data Preprocessing

def extract_glcm_features(image):
    # Convert image to grayscale if it's not already
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    # Calculate GLCM
    distances = [1]  # distance between pixels
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # angles for co-occurrence matrix
    glcm = greycomatrix(gray, distances=distances, angles=angles, symmetric=True, normed=True)
    # Calculate GLCM properties
    contrast = greycoprops(glcm, 'contrast').ravel()
    dissimilarity = greycoprops(glcm, 'dissimilarity').ravel()
    homogeneity = greycoprops(glcm, 'homogeneity').ravel()
    energy = greycoprops(glcm, 'energy').ravel()
    correlation = greycoprops(glcm, 'correlation').ravel()
    return np.hstack([contrast, dissimilarity, homogeneity, energy, correlation])

def load_data(folder_path):
    X = []
    y = []
    for label, class_name in enumerate(['benign', 'malignant']):
        class_folder = os.path.join(folder_path, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            image = cv2.imread(img_path)
            features = extract_glcm_features(image)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)


In [7]:
# Step 2: Load and preprocess training data
train_folder = "/content/extracted_data/archive (4)/train"
X_train, y_train = load_data(train_folder)

/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag

In [8]:
# Step 3: Load and preprocess testing data
test_folder = "/content/extracted_data/archive (4)/test"
X_test, y_test = load_data(test_folder)

/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag

##Model Training

#Implementing Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
# Step 4: Train Logistic Regression model
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
# Step 5: Evaluate model
y_pred_lr = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy)

Accuracy: 0.6556291390728477


In [13]:
from sklearn.metrics import accuracy_score, classification_report

In [14]:
accuracy_score(y_pred_lr,y_test)
print(classification_report(y_pred_lr,y_test))

              precision    recall  f1-score   support

           0       0.60      0.64      0.62        66
           1       0.70      0.67      0.69        85

    accuracy                           0.66       151
   macro avg       0.65      0.65      0.65       151
weighted avg       0.66      0.66      0.66       151



In [15]:
from sklearn.metrics import confusion_matrix

In [16]:
confusion_matrix(y_pred_lr,y_test)

array([[42, 24],
       [28, 57]])

Model selection with GridSearchCV

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
# Define hyperparameters grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}

In [19]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, cv=5)

In [20]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2']})

In [21]:
# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

Best hyperparameters: {'C': 0.001, 'penalty': 'l2'}


In [22]:
# Initialize Logistic Regression with best hyperparameters
lr_tuned = LogisticRegression(**best_params)

In [23]:
# Fit the model with tuned hyperparameters
lr_tuned.fit(X_train, y_train)

LogisticRegression(C=0.001)

In [24]:
y_pred_lr = lr_tuned.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy)

Accuracy: 0.7019867549668874


In [25]:
from sklearn.metrics import accuracy_score, classification_report

In [26]:
accuracy_score(y_pred_lr,y_test)
print(classification_report(y_pred_lr,y_test))

              precision    recall  f1-score   support

           0       0.67      0.68      0.68        69
           1       0.73      0.72      0.72        82

    accuracy                           0.70       151
   macro avg       0.70      0.70      0.70       151
weighted avg       0.70      0.70      0.70       151



In [27]:
from sklearn.metrics import confusion_matrix

In [28]:
confusion_matrix(y_pred_lr,y_test)

array([[47, 22],
       [23, 59]])

#Implementing a Random Forest Classifier

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
rfc=RandomForestClassifier()

In [31]:
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [32]:
y_pred_rfc = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_rfc)
print("Accuracy:", accuracy)

Accuracy: 0.8211920529801324


In [33]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np

In [34]:
accuracy_score(y_pred_rfc,y_test)
print(classification_report(y_pred_rfc,y_test))

              precision    recall  f1-score   support

           0       0.77      0.83      0.80        65
           1       0.86      0.81      0.84        86

    accuracy                           0.82       151
   macro avg       0.82      0.82      0.82       151
weighted avg       0.82      0.82      0.82       151



In [35]:
confusion_matrix(y_pred_rfc,y_test)

array([[54, 11],
       [16, 70]])

Model Selection  with GridSearchCV

In [36]:
# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [37]:
# Create GridSearchCV object
grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='accuracy')

In [38]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False], 'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [39]:
# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'bootstrap': False, 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}


In [40]:
# Print the best estimator found
best_random_forest = grid_search.best_estimator_
print("\nBest Estimator:", best_random_forest)


Best Estimator: RandomForestClassifier(bootstrap=False, max_depth=5, min_samples_leaf=4,
                       min_samples_split=10, n_estimators=300)


In [41]:
y_pred_brf = best_random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_brf)
print("Accuracy:", accuracy)

Accuracy: 0.7417218543046358


In [42]:
from sklearn.metrics import accuracy_score, classification_report

In [43]:
accuracy_score(y_pred_brf,y_test)
print(classification_report(y_pred_brf,y_test))

              precision    recall  f1-score   support

           0       0.59      0.80      0.68        51
           1       0.88      0.71      0.78       100

    accuracy                           0.74       151
   macro avg       0.73      0.76      0.73       151
weighted avg       0.78      0.74      0.75       151



In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
confusion_matrix(y_pred_brf,y_test)

array([[41, 10],
       [29, 71]])

#Implementing a KNN

In [46]:
from sklearn.neighbors import KNeighborsClassifier

In [47]:
knn=KNeighborsClassifier(n_neighbors=7)

In [48]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [49]:
y_pred_knn = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_knn)
print("Accuracy:", accuracy)

Accuracy: 0.7284768211920529


In [50]:
accuracy_score(y_pred_knn,y_test)
print(classification_report(y_pred_knn,y_test))

              precision    recall  f1-score   support

           0       0.60      0.76      0.67        55
           1       0.84      0.71      0.77        96

    accuracy                           0.73       151
   macro avg       0.72      0.74      0.72       151
weighted avg       0.75      0.73      0.73       151



In [51]:
confusion_matrix(y_pred_knn,y_test)

array([[42, 13],
       [28, 68]])

Model selection GridSearchCV

In [52]:
# Set up parameter grid
param_grid = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

In [53]:
# Instantiate KNN Classifier
knn = KNeighborsClassifier()

In [54]:
# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5)

In [55]:
# Fit GridSearchCV
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']})

In [56]:
# Get Best Parameters
best_params = grid_search.best_params_

In [57]:
# Evaluate Model Performance
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)

print("Best Parameters:", best_params)
print("Accuracy:", accuracy)

Best Parameters: {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'distance'}
Accuracy: 0.8675496688741722


In [58]:
y_pred_bknn = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_bknn)
print("Accuracy:", accuracy)

Accuracy: 0.8675496688741722


In [59]:
from sklearn.metrics import accuracy_score, classification_report

In [60]:
accuracy_score(y_pred_bknn,y_test)
print(classification_report(y_pred_bknn,y_test))

              precision    recall  f1-score   support

           0       0.81      0.89      0.85        64
           1       0.91      0.85      0.88        87

    accuracy                           0.87       151
   macro avg       0.86      0.87      0.87       151
weighted avg       0.87      0.87      0.87       151



In [77]:
confusion_matrix(y_pred_bknn,y_test)

array([[57,  7],
       [13, 74]])

#Implementing a Decision Tree

In [78]:
from sklearn.tree import DecisionTreeClassifier

In [79]:
dtc=DecisionTreeClassifier()

In [80]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [81]:
y_pred_dtc = dtc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_dtc)
print("Accuracy:", accuracy)

Accuracy: 0.8344370860927153


In [82]:
accuracy_score(y_pred_dtc,y_test)
print(classification_report(y_pred_dtc,y_test))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82        67
           1       0.86      0.83      0.85        84

    accuracy                           0.83       151
   macro avg       0.83      0.83      0.83       151
weighted avg       0.84      0.83      0.83       151



In [83]:
confusion_matrix(y_pred_dtc,y_test)

array([[56, 11],
       [14, 70]])

model selection with gridsearchCV

In [86]:

# Set up parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None]
}


In [87]:
# Create GridSearchCV object
grid_search = GridSearchCV(dtc, param_grid, cv=5, scoring='accuracy')


In [88]:
# Instantiate Decision Tree Classifier
dt = DecisionTreeClassifier()

In [89]:
# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5)

In [90]:
# Fit GridSearchCV
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30, 40, 50],
                         'max_features': ['auto', 'sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [91]:
# Get Best Parameters
best_params = grid_search.best_params_

In [92]:
# Evaluate Model Performance
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)

print("Best Parameters:", best_params)
print("Accuracy:", accuracy)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5}
Accuracy: 0.7284768211920529


In [93]:
y_pred_bdtc = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_bdtc)
print("Accuracy:", accuracy)

Accuracy: 0.7284768211920529


In [94]:
accuracy_score(y_pred_bdtc,y_test)
print(classification_report(y_pred_bdtc,y_test))

              precision    recall  f1-score   support

           0       0.69      0.72      0.70        67
           1       0.77      0.74      0.75        84

    accuracy                           0.73       151
   macro avg       0.73      0.73      0.73       151
weighted avg       0.73      0.73      0.73       151



In [95]:
confusion_matrix(y_pred_bdtc,y_test)

array([[48, 19],
       [22, 62]])

##Implementing a Naive Bayes classifier

In [96]:
from sklearn.naive_bayes import GaussianNB

In [97]:
nb=GaussianNB()

In [98]:
nb.fit(X_train, y_train)

GaussianNB()

In [99]:
y_pred_nb = nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_nb)
print("Accuracy:", accuracy)

Accuracy: 0.6026490066225165


In [100]:
accuracy_score(y_pred_nb,y_test)
print(classification_report(y_pred_nb,y_test))

              precision    recall  f1-score   support

           0       0.63      0.56      0.59        78
           1       0.58      0.64      0.61        73

    accuracy                           0.60       151
   macro avg       0.60      0.60      0.60       151
weighted avg       0.61      0.60      0.60       151



In [101]:
confusion_matrix(y_pred_nb,y_test)

array([[44, 34],
       [26, 47]])

Model selection GridSearchCV

In [102]:
# Define the hyperparameters to tune (in this case, priors)
param_grid = {
    'priors': [None, [0.25, 0.25, 0.5], [0.2, 0.3, 0.5]] }

In [103]:
# Create GridSearchCV object
grid_search = GridSearchCV(nb, param_grid, cv=5, scoring='accuracy')

In [104]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py", line 267, in fit
    return self._partial_fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py", line 454, in _partial_fit
    raise ValueError("Number of priors must match number of classes.")
ValueError: Number of prio

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'priors': [None, [0.25, 0.25, 0.5], [0.2, 0.3, 0.5]]},
             scoring='accuracy')

In [105]:
# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'priors': None}


In [106]:
# Print the best estimator found
best_naive_bayes = grid_search.best_estimator_
print("\nBest Estimator:", best_naive_bayes)


Best Estimator: GaussianNB()


In [107]:
# Evaluate the best model on the test set
y_pred_bnb = best_naive_bayes.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_bnb)
print("Accuracy:", accuracy)

Accuracy: 0.6026490066225165


In [108]:
accuracy_score(y_pred_bnb,y_test)
print(classification_report(y_pred_bnb,y_test))

              precision    recall  f1-score   support

           0       0.63      0.56      0.59        78
           1       0.58      0.64      0.61        73

    accuracy                           0.60       151
   macro avg       0.60      0.60      0.60       151
weighted avg       0.61      0.60      0.60       151



In [109]:
confusion_matrix(y_pred_bnb,y_test)

array([[44, 34],
       [26, 47]])

##Implementing a Xgboost

In [110]:
import xgboost as xgb

In [111]:
# Define the XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

In [112]:
# Train the classifier
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [113]:
# Make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test)

In [114]:
# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred_xgb)
print("Accuracy:", accuracy)

Accuracy: 0.8278145695364238


In [115]:
accuracy_score(y_pred_xgb,y_test)
print(classification_report(y_pred_xgb,y_test))

              precision    recall  f1-score   support

           0       0.80      0.82      0.81        68
           1       0.85      0.83      0.84        83

    accuracy                           0.83       151
   macro avg       0.83      0.83      0.83       151
weighted avg       0.83      0.83      0.83       151



In [116]:
accuracy_score(y_pred_xgb,y_test)
print(classification_report(y_pred_xgb,y_test))

              precision    recall  f1-score   support

           0       0.80      0.82      0.81        68
           1       0.85      0.83      0.84        83

    accuracy                           0.83       151
   macro avg       0.83      0.83      0.83       151
weighted avg       0.83      0.83      0.83       151



In [117]:
confusion_matrix(y_pred_xgb,y_test)

array([[56, 12],
       [14, 69]])

model selection with GridsearchCV

In [118]:
# Define the hyperparameters to tune
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'n_estimators': [50, 100, 200],
    'min_child_weight': [1, 3, 5]
}

In [119]:
# Create GridSearchCV object
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')

In [120]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [121]:
# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50}


In [122]:
# Print the best estimator found
best_xgb_classifier = grid_search.best_estimator_
print("\nBest Estimator:", best_xgb_classifier)


Best Estimator: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)


In [123]:
# Evaluate the best model on the test set
y_pred_xgb = best_xgb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_xgb)
print("Accuracy:", accuracy)

Accuracy: 0.7483443708609272


In [124]:
accuracy_score(y_pred_xgb,y_test)
print(classification_report(y_pred_xgb,y_test))

              precision    recall  f1-score   support

           0       0.61      0.80      0.69        54
           1       0.86      0.72      0.79        97

    accuracy                           0.75       151
   macro avg       0.74      0.76      0.74       151
weighted avg       0.77      0.75      0.75       151



In [125]:
confusion_matrix(y_pred_xgb,y_test)

array([[43, 11],
       [27, 70]])

##Implementing SVM(Support Vector Machine)

In [126]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [127]:
svm_model = Pipeline([
    ("PCA", PCA(n_components=100)),  # Optional: PCA for dimensionality reduction
    ("SVM", SVC(kernel="rbf", C=10))
])

In [128]:
pca = PCA()

In [129]:
# Fit PCA on the training data and transform both training and testing data
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [130]:
# Define and train your SVM model
svm_model = SVC()
svm_model.fit(X_train_pca, y_train)

SVC()

In [131]:
# Step 4: Model Evaluation
y_pred_svm = svm_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_svm)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.47019867549668876


In [132]:
# Step 5: Model Evaluation Metrics
print(classification_report(y_pred_svm, y_test))

              precision    recall  f1-score   support

           0       0.11      0.31      0.17        26
           1       0.78      0.50      0.61       125

    accuracy                           0.47       151
   macro avg       0.45      0.41      0.39       151
weighted avg       0.66      0.47      0.54       151



In [133]:
confusion_matrix(y_pred_svm,y_test)

array([[ 8, 18],
       [62, 63]])

Model Selection with GridSearchCv

In [134]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [135]:
#Set Up Parameter Grid
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'linear', 'poly', 'sigmoid']}

In [136]:
svm = SVC()

In [137]:
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5) #Instantiate GridSearchCV

In [ ]:
grid_search.fit(X_train, y_train)  #Fit GridSearchCV

In [ ]:
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
 #Evaluate Model Performance

In [ ]:
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)

In [ ]:
# Step: Model Evaluation
y_pred_bsvm = grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_bsvm)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Step 5: Model Evaluation Metrics
print(classification_report(y_pred_bsvm, y_test))

In [ ]:
confusion_matrix(y_pred_bsvm,y_test)

##Implementing a MLP

In [ ]:
from sklearn.neural_network import MLPClassifier


In [ ]:
# Initialize the MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000, random_state=42)

In [ ]:
# Train the model
mlp.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_pred = mlp.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
confusion_matrix(y_pred,y_test)

Model Selection with GridSearchCv

In [ ]:
# Set up parameter grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}

In [ ]:
# Instantiate MLP Classifier
mlp = MLPClassifier()


In [ ]:
# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5)

In [ ]:
# Fit GridSearchCV
grid_search.fit(X_train, y_train)


In [ ]:
# Get Best Parameters
best_params = grid_search.best_params_

In [ ]:
# Evaluate Model Performance
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)

In [ ]:
print("Best Parameters:", best_params)
print("Accuracy:", accuracy)

In [ ]:
# Step: Model Evaluation
y_pred_bmlp = grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_bmlp)
print("Test Accuracy:", test_accuracy)

In [ ]:
confusion_matrix(y_pred_bmlp,y_test)